# This notebook provides interactive widgets to play with the imaging requirements for the SKA1 SDP

This notebook was previously tested with IPython notebook 4.1.0 using Python 2.7. If you have trouble running this notebook, please check version compatibility.

Last run with Jupyter Notebook 5.0.0 running Python 3.5.2

## First, do the necessary set-up (run the code block below, first)

These first few lines below import the IPython definitions and methods that we will use. 
Please Refer to `ipython_api.py` for the implementation:

In [ ]:
from __future__ import print_function
import sys
from ipywidgets import interact, interact_manual, fixed, ToggleButtons, Select, SelectMultiple, Text, Layout
sys.path+=['..']
from sdp_par_model import reports as iapi
from sdp_par_model.config import PipelineConfig
from sdp_par_model.parameters.definitions import *
from sdp_par_model.parameters.definitions import Constants as c
%matplotlib inline

verbose_display     = ['Overview', 'Details', 'Debug']
def toggles(opts, *args, **kwargs): return ToggleButtons(options=opts, *args, **kwargs)
def adjusts(): return Text(placeholder='e.g. blcoal=False Bmax=40*1000 Nsource=100', layout=Layout(width='95%'))

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}

## Using default values, we can compare telescopes side-by-side

In [ ]:
interact_manual(iapi.compare_telescopes_default,
                telescope_1=toggles(Telescopes.available_teles, description="telescope1"),
                telescope_2=toggles(Telescopes.available_teles, description="telescope2"),
                band_1=toggles(Bands.available_bands),
                band_2=toggles(Bands.available_bands),
                pipeline_1=toggles(Pipelines.available_pipelines),
                pipeline_2=toggles(Pipelines.available_pipelines),
                adjusts_1=adjusts(),
                adjusts_2=adjusts(),
                verbosity=toggles(verbose_display));

## Using default values, can compute results for the High Priority Science Objectives

In [ ]:
interact(iapi.evaluate_hpso_optimized,
         hpso_pipe=toggles(HPSOs.available_hpsos),
         verbosity=toggles(verbose_display));

## We can also interactively play with parameters (via sliders)

The first option is automatic updating of results as the sliders are moved. This may be sluggish

In [ ]:
interact(iapi.evaluate_telescope_manual,
         max_baseline=(10000,200000), Nf_max = (1,2**17,1), Nfacet=(1,10,1), Tsnap=(1.2,1800), 
         telescope=toggles(Telescopes.available_teles), band=toggles(Bands.available_bands),
         pipeline=toggles(Pipelines.available_pipelines), verbosity=toggles(verbose_display));

### The second option is manual triggering of recompute events (recommended). 

This allows more conveniently computing elaborate (slow) optimizations and visualizations per computation, as these are only run when required
In this example, *Tsnap and Nfacet* are *automatically* chosen so as to minimize the value of Rflop

In [ ]:
interact_manual(iapi.evaluate_telescope_optimized,
                max_baseline=(10000,200000), Nf_max = (1,256000,1), telescope=toggles(Telescopes.available_teles), 
                band=toggles(Bands.available_bands), pipeline=toggles(Pipelines.available_pipelines),
                verbosity=toggles(verbose_display));

## 1D "parameter sweep" + visualization functionality

Tests a range of values for some parameter and generates a graph for an expression.

In [ ]:
from IPython.display import display, HTML

expression = 'Rflop'
parameter = 'Tobs'
param_val_min = 3600
param_val_max = 360000
number_steps = 99
telescope = Telescopes.SKA1_Low
pipeline = Pipelines.ICAL
expr_scale = c.peta
band=Bands.Low

display(HTML('<font color="blue">Computing the result -- this may take several (tens of) seconds.</font>'))
cfg = PipelineConfig(telescope=telescope, pipeline=pipeline, band=band)
(param_values, results) = cfg.eval_param_sweep_1d(expression_string=expression, parameter_string=parameter, 
                                                   param_val_min=param_val_min, param_val_max=param_val_max, 
                                                   number_steps=number_steps, verbose=False)
header = 'Plotting %s/%g for %s in pipeline %s as a function of %s' % \
   (expression, expr_scale, telescope, pipeline, parameter)
iapi.plot_line_datapoints(header, param_values, np.array(results)/expr_scale, xlabel=parameter, ylabel=expression)

for i in range(len(param_values)):
    print(param_values[i], ",", (results[i])/c.peta)

## 2D "parameter sweep" + visualization functionality

Same as 1D sweep, but varies two parameters to build a 2D graph.

In [ ]:
from IPython.display import display, HTML

telescope = Telescopes.SKA1_Mid
band = Bands.Mid1
pipeline = Pipelines.FastImg
expression = 'Rflop'
parameters = ('Bmax', 'Tobs')
parameter_ranges = ((30000,150000),(0.15,2.0))
number_steps = 4
verbose = False

display(HTML('<font color="blue">Computing the result -- this may take several (tens of) seconds.</font>'))
cfg = PipelineConfig(telescope=telescope, pipeline=pipeline, band=band)
(p1_values, p2_values, results) = cfg.eval_param_sweep_2d(expression_string=expression, parameters=parameters, 
                                                           params_ranges=parameter_ranges, number_steps=number_steps, 
                                                           verbose=verbose)


iapi.plot_2D_surface('%s/1e15 rate vs #Channels & Max Baseline' % expression,
                     p1_values, p2_values, results/1e15, xlabel=parameters[0], 
                     ylabel=parameters[1], contours=np.array((0.5, 1.0)))
iapi.plot_3D_surface('%s/1e15 rate vs #Channels & Max Baseline' % expression,
                     p1_values, p2_values, results/1e15, xlabel=parameters[0], 
                     ylabel=parameters[1], zlabel=expression, contours=np.array((0.5, 1.0)))